# Chunk Initial Conditions
Tim Tyree<br>
8.13.2020

- DONE: use dask to do a run a trivial function with 8 workers an an array of string input that returns True
- DONE: get a list of files to operate on
- DONE: clean up messy stuff left in my folder structure 
- DONE: for the first of ^that list, functionally chunk it into ninths using chunk_600x600_into_ninths(txt,file_name, save_folder)
- TODO: run dask on to create my ic's

In [9]:
#pylab
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt
# from pylab import imshow, show
import time
# import skimage as sk
# from skimage import measure, filters

# import matplotlib.pyplot as plt, numpy as np, os
# from numba import njit, jit, vectorize
# from PIL import Image
# import imageio

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()

import dask.bag as db
from lib.chunk_ic import chunk_600x600_from_fn

#load the libraries
from lib import *

%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Goal: use dask to do a run a trivial function with 8 workers an an array of string input that returns True

## dev initial dask bag use

In [27]:
from lib.chunk_ic import chunk_600x600_into_ninths, load_fortran, parse_input_fn

In [18]:
import dask
# !pip install "dask[bag]" --upgrade
import dask.bag as db

In [21]:
#find file interactively
print("please select a file from within the desired folder.")
file = search_for_file()

#get a list of all files
#change to the directory containing file
folder_name = os.path.dirname(file)
os.chdir(folder_name)

#find folder interactively. Crashes jupyter's root kernel :(
# from tkinter import Tk
# from tkinter.filedialog import askdirectory
# path = askdirectory(title='Select Folder') # shows dialog box and return the path
# print(path) 

please select a file from within the desired folder.
File: /Users/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-1/ic-wj/ic_600x600.122


In [22]:
# get all .csv files in the current working directory
retval = !ls
file_name_list = list(retval)
# check each file if it ends in .csv before merging it
def is_csv(file_name):
    return file_name[-4:]=='.csv'
file_name_list = [f for f in file_name_list if not is_csv(f)]
cwd = os.getcwd()
input_fn_list = [cwd+fn for fn in file_name_list]

In [37]:
# use Dask Bag to load your data. This parallelizes the loading step and reduces inter-worker communication:
# triv = lambda object:object
# load_from_filename = triv
load_from_filename = load_fortran
b = db.from_sequence(input_fn_list).map(load_from_filename)
# b = b.map(triv)
retval = list(b)

In [ ]:
# result = b.compute()
# result = list(b)
# b.to_textfiles('/path/to/data/*.json.gz')

In [ ]:
# use Dask Bag to load your data. This parallelizes the loading step and reduces inter-worker communication:
# triv = lambda object:object
# load_from_filename = triv
load_from_filename = load_fortran
b = db.from_sequence(input_fn_list).map(load_from_filename)
# b = b.map(triv)
retval = list(b)

## for the first of ^that list, functionally chunk it into ninths 
using chunk_600x600_into_ninths(txt,file_name, save_folder)

In [41]:
input_fn = '/Users/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-1/ic-wj/ic_600x600.122'

In [48]:
from dask.delayed import delayed


In [55]:
#use the loading protocol on one file
# d = delayed(load_fortran)(input_fn)

In [6]:
def chunk_from_fn(input_fn):
    arr = load_fortran(input_fn)
    file_name, input_folder, output_folder, tmp_folder = parse_input_fn(input_fn)
    chunk_600x600_into_ninths(txt=arr,
                              file_name=file_name.replace('600','200'),
                              save_folder=output_folder
                             )
    print(f'done with {input_fn}!')
    return True

In [60]:
chunk_from_fn(input_fn)

True

# fill ic-in with ic-wj chunked into ninths

In [7]:
# get input_fn_list interactively
#find file interactively
print("please select a file from within the desired folder.")
file = search_for_file()

#get a list of all files
#change to the directory containing file
folder_name = os.path.dirname(file)
os.chdir(folder_name)

please select a file from within the desired folder.
File: /Users/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-1/ic-wj/ic_600x600.121


In [13]:
# get all ic files that are in the current working directory that are not .csv's
retval = !ls
file_name_list = list(retval)
# check each file if it ends in .csv before merging it
def is_csv(file_name):
    return file_name[-4:]=='.csv'
file_name_list = [f for f in file_name_list if not is_csv(f)]
cwd = os.getcwd()
input_fn_list = ['/'.join((cwd,fn)) for fn in file_name_list]

In [ ]:
# def chunk_600x600_from_fn(input_fn):
# 	input_fn = os.path.abspath(input_fn)
# 	arr = load_fortran(input_fn)
# 	file_name, input_folder, output_folder, tmp_folder = parse_input_fn(input_fn)
# 	chunk_600x600_into_ninths(txt=arr,
# 							  file_name=file_name.replace('600','200'),
# 							  save_folder=output_folder
# 							 )
# 	print(f'done chunking {input_fn}!')
# 	return True

In [14]:
start = time.time()
b = db.from_sequence(input_fn_list).map(chunk_600x600_from_fn)
retval = list(b)
print(f"run time for chunking: {time.time()-start:.2f} [seconds?]")

run time for chunking: 23.21 [seconds?]
